In [27]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
races = pd.read_csv('/content/drive/MyDrive/Colab Datasets/races.csv')
results = pd.read_csv('/content/drive/MyDrive/Colab Datasets/results.csv')
standings = pd.read_csv('/content/drive/MyDrive/Colab Datasets/driver_standings.csv')
status = pd.read_csv('/content/drive/MyDrive/Colab Datasets/status.csv')
drivers = pd.read_csv('/content/drive/MyDrive/Colab Datasets/drivers.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
results = results.drop(columns=['status'], errors='ignore')
results = results.merge(status, on='statusId', how='left')
results = results.merge(races[['raceId', 'year']], on='raceId', how='left')

print(results)


       resultId  raceId  driverId  constructorId number  grid position  \
0             1      18         1              1     22     1        1   
1             2      18         2              2      3     5        2   
2             3      18         3              3      7     7        3   
3             4      18         4              4      5    11        4   
4             5      18         5              1     23     3        5   
...         ...     ...       ...            ...    ...   ...      ...   
26754     26760    1144       825            210     20    14       16   
26755     26761    1144       859            215     30    12       17   
26756     26762    1144       822             15     77     9       \N   
26757     26763    1144       861              3     43    20       \N   
26758     26764    1144       815              9     11    10       \N   

      positionText  positionOrder  points  ...  rank fastestLapTime  \
0                1              1    10.

In [23]:
crashes = results[results['status'].str.contains('Collision|Accident', na=False)]
races = pd.read_csv('races.csv')


print(results.columns)

results = results.drop(columns=['status_x', 'status_y', 'year_x', 'year_y'], errors='ignore')


print(results.columns.tolist())


Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'status_x', 'status_y', 'year_x',
       'year_y', 'status', 'year'],
      dtype='object')
['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid', 'position', 'positionText', 'positionOrder', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId', 'status', 'year']


In [44]:
# Just keep raceId and round from the races file
races_rounds = races[['raceId', 'round']]

# Merge round into crashes
crashes = crashes.merge(races_rounds, on='raceId', how='left')


print(crashes.columns)


Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'status_x', 'status_y', 'year_x',
       'year_y', 'status', 'year', 'round'],
      dtype='object')


In [24]:
crash_counts = crashes.groupby(['driverId', 'year']).size().reset_index(name='crashes')


In [ ]:
crash_counts = crash_counts.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', how='left')
crash_counts['driver_name'] = crash_counts['forename'] + ' ' + crash_counts['surname']


In [46]:
# 🧼 Re-load fresh, unmerged version of standings
standings = pd.read_csv('/content/drive/MyDrive/Colab Datasets/driver_standings.csv')

# ✅ Clean merge with both year and round
standings = standings.merge(races[['raceId', 'year', 'round']], on='raceId', how='left')


In [47]:
print(standings)

       driverStandingsId  raceId  driverId  points  position positionText  \
0                      1      18         1    10.0         1            1   
1                      2      18         2     8.0         2            2   
2                      3      18         3     6.0         3            3   
3                      4      18         4     5.0         4            4   
4                      5      18         5     4.0         5            5   
...                  ...     ...       ...     ...       ...          ...   
34858              73266    1144       858     0.0        23           23   
34859              73267    1144       860     7.0        18           18   
34860              73268    1144       861     5.0        19           19   
34861              73269    1144       859     4.0        21           21   
34862              73270    1144       862     0.0        24           24   

       wins  year  round  
0         1  2008      1  
1         0  2008    

In [36]:

last_races = races.groupby('year')['round'].max().reset_index()
final_race_ids = races.merge(last_races, on=['year', 'round'], how='inner')[['year', 'raceId']]


final_standings = standings.merge(final_race_ids, on=['year', 'raceId'], how='inner')
final_standings = final_standings[['driverId', 'year', 'position']]
final_standings.rename(columns={'position': 'final_position'}, inplace=True)

In [37]:
print (final_standings)

      driverId  year  final_position
0            1  2008               1
1            2  2008               6
2            3  2008              13
3            4  2008               5
4            5  2008               7
...        ...   ...             ...
3187       858  2024              23
3188       860  2024              18
3189       861  2024              19
3190       859  2024              21
3191       862  2024              24

[3192 rows x 3 columns]


In [38]:
crash_data = crash_counts.merge(final_standings, on=['driverId', 'year'], how='left')


In [39]:
print(crash_data)

      driverId  year  crashes forename    surname       driver_name  \
0            1  2008        1    Lewis   Hamilton    Lewis Hamilton   
1            1  2009        2    Lewis   Hamilton    Lewis Hamilton   
2            1  2010        2    Lewis   Hamilton    Lewis Hamilton   
3            1  2011        2    Lewis   Hamilton    Lewis Hamilton   
4            1  2012        3    Lewis   Hamilton    Lewis Hamilton   
...        ...   ...      ...      ...        ...               ...   
1221       856  2023        2     Nyck   de Vries     Nyck de Vries   
1222       857  2023        1    Oscar    Piastri     Oscar Piastri   
1223       858  2023        3    Logan   Sargeant    Logan Sargeant   
1224       858  2024        2    Logan   Sargeant    Logan Sargeant   
1225       861  2024        2   Franco  Colapinto  Franco Colapinto   

      final_position  
0                1.0  
1                5.0  
2                4.0  
3                5.0  
4                4.0  
...      

In [48]:
crash_standings = crashes.merge(
    standings,
    on=['driverId', 'year'],
    how='left',
    suffixes=('', '_standing')  # prevent suffix clashes
)


In [50]:
crash_standings = crash_standings[crash_standings['round_standing'] < crash_standings['round']]


In [51]:
crash_standings = crash_standings.sort_values(
    by=['driverId', 'raceId', 'round_standing'], ascending=[True, True, False]
)



In [52]:
pre_crash = crash_standings.groupby(['driverId', 'raceId']).first().reset_index()


In [53]:
pre_crash.rename(columns={'position': 'pre_crash_position'}, inplace=True)


In [55]:
print(pre_crash[['driverId', 'raceId', 'year', 'round', 'pre_crash_position']])



      driverId  raceId  year  round pre_crash_position
0            1      12  2009     12                 \N
1            1      13  2009     13                 12
2            1      24  2008      7                 \N
3            1     350  2010     14                 \N
4            1     351  2010     15                 \N
...        ...     ...   ...    ...                ...
1690       858    1114  2023     16                 \N
1691       858    1126  2024      6                 \N
1692       858    1129  2024      9                 \N
1693       861    1141  2024     21                 \N
1694       861    1143  2024     23                 \N

[1695 rows x 5 columns]


In [56]:
final_rounds = standings.groupby(['driverId', 'year'])['round'].max().reset_index()


In [57]:
final_standings = standings.merge(final_rounds, on=['driverId', 'year', 'round'], how='inner')
final_standings.rename(columns={'position': 'final_position'}, inplace=True)


In [58]:
pre_crash_vs_final = pre_crash.merge(
    final_standings[['driverId', 'year', 'final_position']],
    on=['driverId', 'year'],
    how='left'
)


In [62]:
pre_crash_vs_final['pre_crash_position'] = pd.to_numeric(pre_crash_vs_final['pre_crash_position'], errors='coerce')
pre_crash_vs_final['final_position'] = pd.to_numeric(pre_crash_vs_final['final_position'], errors='coerce')


In [63]:
pre_crash_vs_final['standing_drop'] = (pre_crash_vs_final['pre_crash_position']) - (pre_crash_vs_final['final_position'])


In [65]:
drivers = pd.read_csv('/content/drive/MyDrive/Colab Datasets/drivers.csv')

drivers['driver_name'] = drivers['forename'] + ' ' + drivers['surname']

pre_crash_vs_final = pre_crash_vs_final.merge(drivers[['driverId', 'driver_name']], on='driverId', how='left')


In [67]:
print(pre_crash_vs_final[['driver_name', 'year', 'round', 'pre_crash_position', 'final_position', 'standing_drop']])


           driver_name  year  round  pre_crash_position  final_position  \
0       Lewis Hamilton  2009     12                 NaN               5   
1       Lewis Hamilton  2009     13                12.0               5   
2       Lewis Hamilton  2008      7                 NaN               1   
3       Lewis Hamilton  2010     14                 NaN               4   
4       Lewis Hamilton  2010     15                 NaN               4   
...                ...   ...    ...                 ...             ...   
1690    Logan Sargeant  2023     16                 NaN              21   
1691    Logan Sargeant  2024      6                 NaN              23   
1692    Logan Sargeant  2024      9                 NaN              23   
1693  Franco Colapinto  2024     21                 NaN              19   
1694  Franco Colapinto  2024     23                 NaN              19   

      standing_drop  
0               NaN  
1               7.0  
2               NaN  
3          

In [71]:
final_clean1 = pre_crash_vs_final.dropna(subset=['pre_crash_position', 'final_position'])


In [73]:
final_clean1 = final_clean.sort_values('standing_drop')


In [74]:
print(final_clean1)

      driverId  raceId  resultId  constructorId number  grid  \
1017       208     554     13569              1     36    26   
960        197     619     15300             37     18    17   
1158       239     570     14010             63     18    16   
752        148     324      7123             46     35    20   
1362       438     720     17427             92     24    19   
...        ...     ...       ...            ...    ...   ...   
569         95     330      7343              6      2     5   
496         77     330      7342              1     28     3   
891        178     490     11773              3      1     9   
1631       844    1029     24603              6     16    14   
7            1     867     21418              1      4     2   

      pre_crash_position positionText  positionOrder  points  ...  \
1017                17.0           17             17     0.0  ...   
960                 11.0           11             11     0.0  ...   
1158                10.0

In [75]:
final_clean1.to_csv('f1_crash_impact_study.csv', index=False)


In [77]:
# This already has driverId, year, and final_position
champions = final_standings[final_standings['final_position'] == 1][['year', 'driverId']]
champions = champions.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', how='left')
champions['champion_name'] = champions['forename'] + ' ' + champions['surname']
champions = champions[['year', 'champion_name']]


In [78]:
crashgate = pre_crash_vs_final.copy()
crashgate = crashgate.merge(champions, on='year', how='left')


In [80]:
crashgate_final = crashgate[[
    'driver_name', 'year', 'round', 'pre_crash_position',
    'standing_drop', 'final_position', 'champion_name'
]]


In [81]:
crashgate_final.to_csv('crashgate.csv', index=False)


In [82]:
from google.colab import files
files.download('crashgate.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>